In [1]:
%cd /home/mrearle/repos/R2R-EnvDrop-ObjAttn/

/mnt-homes/kraken/mrearle/repos/R2R-EnvDrop-ObjAttn


# Imports

In [2]:
import sys
import torch

metadata_parser_path = '/home/mrearle/repos/R2R-EnvDrop-ObjAttn/metadata_parser'
if metadata_parser_path not in sys.path:
    sys.path.append(metadata_parser_path)

mattersim_path = '/home/mrearle/datasets/Matterport3DSimulator/build'
if mattersim_path not in sys.path:
    sys.path.append(mattersim_path)
    
src_path = '/home/mrearle/repos/R2R-EnvDrop-ObjAttn/r2r_src'
if src_path not in sys.path:
    sys.path.append(src_path)

from collections import defaultdict

In [4]:
from parse_house_segmentations import HouseSegmentationFile

# Setup and run agent

In [5]:
from visualization_v2.agent_utils import load_args, args, load_envs, setup_agent as _setup_agent
import os

def setup_agent(
    base_name: str,
    max_obj_number: int = 20,
    obj_aux_task: bool = True,
    include_objs: bool = True,
    reduced_envs: bool = True,
    dataset: str = "R2R",
):
    print("Parsing args")
    load_args(
        max_obj_number=max_obj_number,
        obj_aux_task=obj_aux_task,
        include_objs=include_objs,
        reduced_envs=reduced_envs,
        dataset=dataset,
    )

    print("Loading envs")
    train_env, val_envs, tok = load_envs()

    experiment = []
    if args.dataset.upper() != "R2R":
        experiment.append(args.dataset)
    if args.max_obj_number != 20:
        experiment.append(f"obj({args.max_obj_number})")
    if args.obj_aux_task:
        experiment.append(f"aux({args.obj_aux_task_weight})")
    if args.reduced_envs:
        experiment.append(f"reduced")

    args.experiment = "_".join(experiment) or "default"
    print(base_name, args.experiment)
    args.save_dir = os.path.join(base_name, args.experiment, "__")
    load = f"snap/{base_name}/{args.experiment}/state_dict/best_val_unseen"

    agent, obj_attns, view_attns = _setup_agent(train_env, tok, load)

    agent.results_path = f"results/{base_name}/{args.experiment}.json"

    return agent, val_envs

Optimizer: Using RMSProp
Using device: cuda
_CudaDeviceProperties(name='GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11178MB, multi_processor_count=28)


	Training model default in experiment default


Namespace(CANDIDATE_FEATURES='img_features/ResNet-152-candidate.tsv', IMAGENET_FEATURES='img_features/ResNet-152-imagenet.tsv', OBJECT_CLASS_FILE='/workspace1/mrearle/object_classes.json', OBJECT_FEATURES='/workspace1/mrearle/object_features_filtered.hdf5', TRAINVAL_VOCAB='tasks/R2R/data/trainval_vocab.txt', TRAIN_VOCAB='tasks/R2R/data/train_vocab.txt', accumulate_grad=False, aemb=64, alpha=0.5, angle_feat_size=4, attn='soft', aug=None, batchSize=64, beam=False, bidir=True, buffer_objs=False, candidate_mask=False, candidates=1, dataset='R2R', device=device(type='cuda'), dropout=0.5, encode='word', epsilon=0.1, experiment='default', fast_train=False, featdropout=0.3, feature_size=2048, features='imagenet', features_fast='img_features/ResNet-152-imagenet-fast.tsv', feedback='sample

In [3]:
%%capture cap
from importlib import reload
import visualization_v2.agent_utils as agent_utils
reload(agent_utils)
setup_and_run_agent = agent_utils.setup_and_run_agent


In [4]:
agent, _, _ = setup_and_run_agent(
    base_name="base",
    max_obj_number=20,
    obj_aux_task = False,
    include_objs = False,
    reduced_envs = True,
    dataset = "R2R",
)

Parsing args
Namespace(CANDIDATE_FEATURES='img_features/ResNet-152-candidate.tsv', IMAGENET_FEATURES='img_features/ResNet-152-imagenet.tsv', OBJECT_CLASS_FILE='/workspace1/mrearle/object_classes.json', OBJECT_FEATURES='/workspace1/mrearle/object_features_filtered.hdf5', TRAINVAL_VOCAB='tasks/R2R/data/trainval_vocab.txt', TRAIN_VOCAB='tasks/R2R/data/train_vocab.txt', accumulate_grad=False, aemb=64, alpha=0.5, angle_feat_size=128, attn='soft', aug=None, batchSize=64, beam=False, bidir=True, buffer_objs=False, candidate_mask=False, candidates=1, dataset='R2R', device=device(type='cuda'), dropout=0.5, encode='word', epsilon=0.1, experiment='reduced', fast_train=False, featdropout=0.3, feature_size=2048, features='imagenet', features_fast='img_features/ResNet-152-imagenet-fast.tsv', feedback='sample', gamma=0.9, ignoreid=-100, include_objs=False, include_objs_lstm=False, iters=100000, listener=None, load='/home/mrearle/repos/R2R-EnvDrop-ObjAttn/snap/agent_base-reduced/state_dict/best_val_un

In [1]:
args.save_dir

NameError: name 'args' is not defined

In [12]:
import json

In [10]:
for env_name, (env, evaluator) in val_envs.items():
    agent.logs = defaultdict(list)
    agent.env = env

    iters = None
    agent.test(use_dropout=False, feedback="argmax", iters=iters)

result = agent.get_results()
score_summary, _ = evaluator.score(result)
loss_str = "Env name: %s" % env_name
for metric, val in score_summary.items():
    loss_str += ", %s: %.4f" % (metric, val)
print(loss_str, flush=True)
# log_dir = "results/submit/%s" % args.save_dir
# json.dump(
#     result,
#     open(os.path.join(log_dir, "submit_%s.json" % env_name), "w"),
#     sort_keys=True,
#     indent=4,
#     separators=(",", ": "),
# )
# print(evaluator.score(agent.get_results())[0])
# agent_result = agent.write_results()



NameError: name 'val_envs' is not defined

In [16]:
evaluator.splits

['val_unseen']

# Compare trajectory data

In [1]:
import json
import networkx as nx
import numpy as np

In [2]:
ground_truth_path = 'semantically_richer_instructions/R2R_craft_val_unseen.json'
base_agent_path = 'results/base/craft_reduced/best_val_unseen.json'
obj_agent_path = 'results/obj/craft_obj(32)_aux(0.1)_reduced/best_val_unseen.json'

In [3]:
SMALL_SPLITS = {
    "train": [
        "Uxmj2M2itWa",
        "82sE5b5pLXE",
        "2n8kARJN3HM",
        "1pXnuDYAj8r",
        "VLzqgDo317F",
        "p5wJjkQkbXX",
        "r1Q1Z4BcV1o",
        "HxpKQynjfin",
        "PuKPg4mmafe",
        "cV4RVeZvu5T",
        "PX4nDJXEHrG",
        "VFuaQ6m2Qom",
        "JF19kD82Mey",
        "sT4fr6TAbpF",
        "E9uDoFAP3SH",
        "XcA2TqTSSAj",
        "8WUmhLawc2A",
        "EDJbREhghzL",
        "1LXtFkjw3qL",
        "pRbA3pwrgk9",
        "gZ6f7yhEvPG",
    ],
    "val_seen": [
        "Uxmj2M2itWa",
        "82sE5b5pLXE",
        "2n8kARJN3HM",
        "1pXnuDYAj8r",
        "VLzqgDo317F",
        "p5wJjkQkbXX",
        "r1Q1Z4BcV1o",
        "PuKPg4mmafe",
        "cV4RVeZvu5T",
        "PX4nDJXEHrG",
        "VFuaQ6m2Qom",
        "JF19kD82Mey",
        "sT4fr6TAbpF",
        "E9uDoFAP3SH",
        "XcA2TqTSSAj",
        "8WUmhLawc2A",
        "1LXtFkjw3qL",
        "EDJbREhghzL",
        "pRbA3pwrgk9",
        "gZ6f7yhEvPG",
    ],
    "val_unseen": [
        "2azQ1b91cZZ",
        "QUCTc6BB5sX",
        "zsNo4HB9uLZ",
        "oLBMNvg9in8",
        "8194nk5LbLH",
        "EU6Fwq7SyZv",
        "x8F5xyUWy9e",
        "Z6MFQCViBuw",
        "X7HyMhZNoso",
        "pLe4wQe7qrG",
        "TbHJrupSAjP",
    ],
}

scans = {x for split in SMALL_SPLITS.values() for x in split}

In [4]:
def load_nav_graphs(scans):
    """Load connectivity graph for each scan"""

    def distance(pose1, pose2):
        """Euclidean distance between two graph poses"""
        return (
            (pose1["pose"][3] - pose2["pose"][3]) ** 2
            + (pose1["pose"][7] - pose2["pose"][7]) ** 2
            + (pose1["pose"][11] - pose2["pose"][11]) ** 2
        ) ** 0.5

    graphs = {}
    for scan in scans:
        with open("connectivity/%s_connectivity.json" % scan) as f:
            G = nx.Graph()
            positions = {}
            data = json.load(f)
            for i, item in enumerate(data):
                if item["included"]:
                    for j, conn in enumerate(item["unobstructed"]):
                        if conn and data[j]["included"]:
                            positions[item["image_id"]] = np.array([item["pose"][3], item["pose"][7], item["pose"][11]])
                            assert data[j]["unobstructed"][i], "Graph should be undirected"
                            G.add_edge(
                                item["image_id"],
                                data[j]["image_id"],
                                weight=distance(item, data[j]),
                            )
            nx.set_node_attributes(G, values=positions, name="position")
            graphs[scan] = G
    
    img_id_to_scan = {}
    for scan, G in graphs.items():
        for img_id in list(G.nodes()):
            img_id_to_scan[img_id] = scan
    return graphs, img_id_to_scan

In [7]:
graphs, img_id_to_scan = load_nav_graphs(scans)
ERROR_MARGIN = 3.0
distances = {}
for scan, G in graphs.items():  # compute all shortest paths
    distances[scan] = dict(nx.all_pairs_dijkstra_path_length(G))

In [8]:
def success(vp1, vp2):
    scan = img_id_to_scan[vp1]
    pos1 = graphs[scan].nodes[vp1]['position']
    pos2 = graphs[scan].nodes[vp2]['position']
    dist = distances[scan][vp1][vp2]
    # dist = np.linalg.norm(pos1 - pos2)
    if dist <= 3.0:
        return True
    else:
        # print(f"{vp1},{vp2},{dist}")
        return False

In [9]:
with open(ground_truth_path) as f:
    ground_truth = json.load(f)
ground_truth = {v['path_id']: v['path'] for v in ground_truth}

with open(base_agent_path) as f:
    base_agent = json.load(f)
base_agent = {v['instr_id']: v['trajectory'] for v in base_agent}

with open(obj_agent_path) as f:
    obj_agent = json.load(f)
obj_agent = {v['instr_id']: v['trajectory'] for v in obj_agent}

In [10]:
succs = {'base': set(), 'obj': set()}
fails = {'base': set(), 'obj': set()}

for instr_id in base_agent:
    path_id, _ = instr_id.split('_')

    base_agent_success = success(base_agent[instr_id][-1][0], ground_truth[int(path_id)][-1])
    obj_agent_success = success(obj_agent[instr_id][-1][0], ground_truth[int(path_id)][-1])

    base_dict = succs if base_agent_success else fails
    obj_dict = succs if obj_agent_success else fails

    base_dict['base'].add(instr_id)
    obj_dict['obj'].add(instr_id)
    

In [15]:
'254_0' in succs['obj']

False

In [26]:
print(f"Successes:\n\tBase: {len(succs['base'])}\n\tObj: {len(succs['obj'])}")
print(f"Fails:\n\tBase: {len(fails['base'])}\n\tObj: {len(fails['obj'])}")
print(f"Success Rate:\n\tBase: {len(succs['base']) / len(base_agent)}\n\tObj: {len(succs['obj']) / len(obj_agent)}")

Successes:
	Base: 389
	Obj: 410
Fails:
	Base: 394
	Obj: 373
Success Rate:
	Base: 0.49680715197956576
	Obj: 0.5236270753512133


In [27]:
obj_better = succs['obj'].intersection(fails['base'])
obj_worse = fails['obj'].intersection(succs['base'])
bot_succ = succs['obj'].intersection(succs['base'])
bot_fail = fails['obj'].intersection(fails['base'])
print(f'Obj succeded where base failed: {len(obj_better)}')
print(f'Obj failed where base succeded: {len(obj_worse)}')
print(f'Both succeded: {len(bot_succ)}')
print(f'Both failed: {len(bot_fail)}')

Obj succeded where base failed: 100
Obj failed where base succeded: 79
Both succeded: 310
Both failed: 294


In [34]:
(79 +  310) / (100 + 310 + 79 + 294)

0.49680715197956576

In [13]:
from itertools import zip_longest
def compact_traj(traj):
    res = []
    for view in traj:
        if view not in res:
            res.append(view)
    return res

In [ ]:
list(bot_fail)[0]

In [35]:
i=0
instr = '4976_0'
print(instr in succs['obj'], instr in succs['base'])
gt = compact_traj(ground_truth[int(instr.split('_')[0])])
base = compact_traj([x[0] for x in base_agent[instr]])
obj = compact_traj([x[0] for x in obj_agent[instr]])

width = 16
gt = [x[:width] for x in gt]
base = [x[:width] for x in base]
obj = [x[:width] for x in obj]

print(f"{'ground truth': <16} {'  base': <16} {'    obj': <16}")
print(*['   '.join(x) for x in zip_longest(gt, base, obj, fillvalue=' '*width)], sep='\n')

True False
ground truth       base               obj         
273092f839e945eb   273092f839e945eb   273092f839e945eb
7ef2af7db85e4b9b   7ef2af7db85e4b9b   7ef2af7db85e4b9b
76e1ef125ea2458c   76e1ef125ea2458c   76e1ef125ea2458c
7976fb70011347e5                      c644381db1ea4bdb
af1005e4b2804e39                      af1005e4b2804e39
f320ae084f3a447d                      f320ae084f3a447d
e8ae2f7717b54c6d                      e8ae2f7717b54c6d


In [ ]:
distances[img_id_to_scan['0a9f30bd318e40de89f71e4bf6987358']]['ac3dc08c7a2646b991fda42ccc42bc47']['0cf8775a4e474671bc23337c84da1540']

In [29]:
with open('/home/mrearle/repos/compute_conv_feats/splitted_instructions_val_unseen.json', 'r') as f:
    obj_split_instruction = json.load(f)

In [30]:
many_obj = { 'success': [], 'fail': [] }
less_obj = { 'success': [], 'fail': [] }
many_obj_base = { 'success': [], 'fail': [] }
less_obj_base = { 'success': [], 'fail': [] }


for instr_id in base_agent:
    path_id, _ = instr_id.split('_')

    path_type = 'many_obj' if instr_id in obj_split_instruction['many_objs'] else 'less_obj'

    base_agent_success = success(base_agent[instr_id][-1][0], ground_truth[int(path_id)][-1])
    obj_agent_success = success(obj_agent[instr_id][-1][0], ground_truth[int(path_id)][-1])

    if path_type == 'many_obj':
        many_obj['success' if obj_agent_success else 'fail'].append(instr_id)
    else:
        less_obj['success' if obj_agent_success else 'fail'].append(instr_id)

    
    if path_type == 'many_obj':
        many_obj_base['success' if base_agent_success else 'fail'].append(instr_id)
    else:
        less_obj_base['success' if base_agent_success else 'fail'].append(instr_id)


In [31]:
print('Obj Agent')
print(len(many_obj['success']), len(many_obj['fail']), len(less_obj['success']), len(less_obj['fail']))
print('Many obj SR:', len(many_obj['success']) / (len(many_obj['success']) + len(many_obj['fail'])))
print('Less obj SR:', len(less_obj['success']) / (len(less_obj['success']) + len(less_obj['fail'])))

Obj Agent
200 243 210 130
Many obj SR: 0.45146726862302483
Less obj SR: 0.6176470588235294


In [32]:
print('Base Agent')
print(len(many_obj_base['success']), len(many_obj_base['fail']), len(less_obj_base['success']), len(less_obj_base['fail']))
print('Many obj SR:', len(many_obj_base['success']) / (len(many_obj_base['success']) + len(many_obj_base['fail'])))
print('Less obj SR:', len(less_obj_base['success']) / (len(less_obj_base['success']) + len(less_obj_base['fail'])))

Base Agent
195 248 194 146
Many obj SR: 0.4401805869074492
Less obj SR: 0.5705882352941176
